<a href="https://colab.research.google.com/github/beatrizolegario/SEMINARIO_PLN/blob/main/Corrected_ParsingDocsUsingLLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**SEMINÁRIO:** Extração de Dados de Documentos Usando LLMs
_Artigo Original_: [Document Parsing Using Large Language Models — With Code](https://towardsdatascience.com/document-parsing-using-large-language-models-with-code-9229fda09cdf)

_Notebook Original_: [Extract_Metadata_With_Large_Language_Models](https://github.com/keitazoumana/LLMs/blob/main/Extract_Metadata_With_Large_Language_Models_V2.ipynb)


- _Autor: Zoumana Keita_
- Revisão dos Dicentes:
  - Beatriz Santos Olegario
  - Bruno JM de Camargo
  -Thiago Bibiano Silva
  


A extração de dados de documentos complexos, como visto no artigo científico abaixo, teve como única alternativa por algum tempo o uso de expressões regulares. Hoje, com os modelos de linguagem de grande porte, surge uma nova perspectiva para essa tarefa. No artigo de Zoumana Keita, são discutidas as vantagens dos LLMs em relação às expressões regulares, com uma implementação utilizando o modelo GPT-4 da OpenAI.



![Descrição da Imagem](https://cdn-images-1.readmedium.com/v2/resize:fit:800/1*glQYLm0bwalGhBgppAbbSg.png)


_Imagem 1: Exemplo de entidadades para serem extraídas de um artigo_


## Comparativo entre Regex e LLM

### Expressões Regulares (Regex)
**Vantagens:**
- Rápido para padrões simples.
- Eficiente com estruturas de documentos bem definidas.

**Desvantagens:**
- Baixa flexibilidade; requer padrões específicos para cada estrutura de documento.
- Não compreende o contexto ou significado do texto.
- Difícil manutenção; exige atualizações constantes para novos formatos.
- Dificuldade em lidar com documentos complexos e variáveis.
- Requer conhecimento técnico para criar padrões eficazes.

### Modelos de Linguagem de Grande Porte (LLMs)
**Vantagens:**
- Alta flexibilidade; adapta-se a diversas estruturas de documentos.
- Compreensão do significado do texto, permitindo melhor extração de dados.
- Fácil manutenção; adapta-se a novos tipos de documentos com mínimas alterações.
- Eficaz em lidar com complexidade e variabilidade.
- Mais intuitivo, utilizando prompts descritivos.

**Desvantagens:**
- Pode ser mais lento devido à complexidade do modelo.
- Requer recursos computacionais robustos.

---


# Implementação

![Descrição da Imagem](https://cdn-images-1.readmedium.com/v2/resize:fit:800/1*ZZR9KMsYHkZdbdoXe-e3bQ.png)

_Imagem 2: Proposta de fluxo de extração de dados propostoto por Keita_

Os principais pontos da implementação proposta por Keita são:
- Um **extrator de textos** à partir de PDF de artigos científicos com uso de OCR para leitura de imagens
- Um **prompt de extração de metadados**, executado usando GPT-4
- Estutura de arquivos abaixo:
```
project
   |
   |---Extract_Metadata_With_Large_Language_Models.ipynb
   |
  data
   |
   |---- extracted_metadata/
   |---- 1706.03762v7.pdf
   |---- prompts
           |
           |------ scientific_papers_prompt.txt
```



**Modificações:**

- **Extrator de Textos**: Não conseguimos implementar a função original com OCR, então:
  - Simplificamos para apenas extrair textos sem apoio de OCR com o uso da lib `PyPDF2`
  - Limitamos a coleta para apenas a primeira página, já que isso bastaria para o caso de uso proposto (Coleta de Autores e Resumo)
- **Extrator de Metadados**: Nâo modificamos o prompt, mas como a procura estava apenas em texto, resolvemos utilizar  o GPT-40-mini para reduzir custos.
  - No exemplo abaixo uma execução com GPT-4o teve custo aproximado de 0.2 Dolar (US) enquanto para a versão mini o custo foi de >0.01 Dolar (US)
- **Estutura de Arquivos**: Emulamos a estrutura de pastas e subida de arquivos no código, criando as pastas quando necessário.





## Instalndo bibliotecas necessárias

Bibliotecas comentadas estavam no artigo original ou não foram usadas devido à simplificação do extrator de texto ou já estavam ociosas no notebook original.

In [ ]:
%%bash

# Instala bibliotecas necessárias
pip install pdfminer.six
#pip install pillow-heif==0.3.2
#pip install matplotlib
#pip install unstructured-inference
#pip install unstructured-pytesseract
#pip install tesseract-ocr
#pip install unstructured
#pip install pi-heif
pip install openai
pip install PyPDF2


# Instala pacotes de OCR e utilitários do sistema necessários
#apt install -V tesseract-ocr
#apt install -V libtesseract-dev

#sudo apt-get update
#apt-get install -V poppler-utils


## Classe ResearchPaperProcessor

Para facilitar a analise do código e modificações necessárias consolidamos as funções presentes no notebook e outras criadas para facilitar a abstração da classe.


1. **`__init__(self, model_id: str)`**
   - Inicialização:
     - Define `model_id` e configura o cliente OpenAI.

2. **`get_client(self)`**
   - Configuração de API:
     - Obtém a chave da API do Google Colab.
     - Configura o cliente OpenAI com a chave.

3. **`read_prompt(prompt_path: str) -> str`**
   - Leitura de Prompt:
     - Abre o arquivo de texto na localização `prompt_path`.
     - Retorna o conteúdo como uma string.

4. **`extract_text_from_pdf(pdf_path: str) -> str`**
   - Extração de Texto:
     - Abre o PDF em `pdf_path`.
     - Utiliza `PyPDF2` para ler o texto da primeira página.
     - Retorna o texto extraído.

5. **`completion_with_backoff(self, **kwargs)`**
   - Chamada à API com Backoff:
     - Faz uma chamada à API OpenAI.
     - Utiliza backoff exponencial para lidar com limites de taxa.

6. **`extract_metadata(self, content: str, prompt_path: str) -> dict`**
   - Preparação:
     - Lê o prompt de `prompt_path`.
   - Chamada GPT:
     - Realiza a chamada à API para extração de metadados.
   - Processamento de Resposta:
     - Limpa e formata a resposta JSON.
     - Corrige erros de análise de JSON, se necessário.

7. **`process_research_paper(self, pdf_path: str, prompt_path: str, output_folder: str)`**
   - Extração de Texto:
     - Extrai o texto do PDF.
   - Extração de Metadados:
     - Chama `extract_metadata` para obter metadados.
   - Salvamento de Resultados:
     - Salva os metadados extraídos em um arquivo JSON no `output_folder`.

In [ ]:
# Importação das bibliotecas
import os
import re
import json
import openai
from pathlib import Path
from PyPDF2 import PdfReader
from tenacity import retry, wait_random_exponential, stop_after_attempt

from openai import OpenAI
from google.colab import userdata

# Importação de bibliotecas não usadas no código adpatado
 #from unstructured.partition.pdf import partition_pdf



class ResearchPaperProcessor:
    """
    Classe para processar artigos de pesquisa em PDF, extrair texto e metadados usando a API OpenAI.
    """

    def __init__(self, model_id: str):
        self.model_id = model_id
        self.client = self.get_client()

    @staticmethod
    def get_client(self):
        """
        Retorna o cliente OpenAI.]
        """
        OPENAI_API_KEY = userdata.get('OPEN_AI_KEY')
        os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

        return OpenAI(api_key = os.environ["OPENAI_API_KEY"])

    @staticmethod
    def read_prompt(prompt_path: str) -> str:
        """
        Lê o prompt para análise de artigos de pesquisa de um arquivo de texto.
        """
        with open(prompt_path, "r") as f:
            return f.read()

    @staticmethod
    def extract_text_from_pdf(pdf_path: str) -> str:
        """
        Extrai texto de um arquivo PDF.
        """
        with open(pdf_path, 'rb') as file:
            reader = PdfReader(file)
            text = ""
            page = reader.pages[0]
            text += page.extract_text() + "\n"
        return text

    @retry(wait=wait_random_exponential(min=1, max=120), stop=stop_after_attempt(10))
    def completion_with_backoff(self, **kwargs):
        """
        Realiza uma chamada de conclusão com a API OpenAI usando backoff exponencial.
        """
        return client.chat.completions.create(**kwargs)

    def extract_metadata(self, content: str, prompt_path: str) -> dict:
        """
        Usa o modelo GPT para extrair metadados do conteúdo do artigo de pesquisa com base no prompt fornecido.
        """
        prompt_data = self.read_prompt(prompt_path)

        try:
            response = self.completion_with_backoff(
                model=self.model_id,
                messages=[
                    {"role": "system", "content": prompt_data},
                    {"role": "user", "content": content}
                ],
                temperature=0.2,
            )

            response_content = response.choices[0].message.content
            if not response_content:
                print("Resposta vazia do modelo")
                return {}

            # Remove os indicadores de blocos de código markdown
            response_content = re.sub(r'```json\s*', '', response_content)
            response_content = re.sub(r'\s*```', '', response_content)

            # Tenta fazer parsing do JSON
            try:
                return json.loads(response_content)
            except json.JSONDecodeError as e:
                print(f"Falha ao analisar o JSON: {e}")
                print(f"Resposta bruta: {response_content}")

                # Tenta extrair JSON da resposta
                match = re.search(r'\{.*\}', response_content, re.DOTALL)
                if match:
                    try:
                        return json.loads(match.group(0))
                    except json.JSONDecodeError as jde:
                        print(f"Falha ao extrair JSON válido da resposta: {jde}")

                return {}

        except Exception as e:
            print(f"Erro ao chamar a API OpenAI: {e}")
            return {}

    def process_research_paper(self, pdf_path: str, prompt_path: str, output_folder: str):
        """
        Processa um único artigo de pesquisa através de todo o pipeline.
        """
        print(f"Processando artigo de pesquisa: {pdf_path}")

        try:
            # Passo 1: Extrair o conteúdo do texto do PDF
            content = self.extract_text_from_pdf(pdf_path)
            print(f"Conteúdo do texto extraído do PDF: {pdf_path}")

            # Passo 2: Extrair metadados usando o modelo GPT
            metadata = self.extract_metadata(content, prompt_path)
            if not metadata:
                print(f"Falha ao extrair metadados para {pdf_path}")
                return
            print(f"Metadados extraídos usando {self.model_id} para {pdf_path}")

            # Passo 3: Salvar o resultado como um arquivo JSON
            output_filename = Path(pdf_path).stem + '.json'
            output_path = os.path.join(output_folder, output_filename)

            with open(output_path, 'w') as f:
                json.dump(metadata, f, indent=2)
            print(f"Metadados salvos em {output_path}")

        except Exception as e:
            print(f"Erro ao processar {pdf_path}: {e}")

## Estrutura do Prompt para Extração de Metadados

1. **Espaço Reservado para o Documento:**
   - `{document}` indica onde o texto completo do documento será inserido para análise.

2. **Atribuição de Papel:**
   - Define o modelo como um "especialista em análise de artigos científicos" para focar na tarefa.

3. **Instruções de Extração:**
   - Lista as propriedades a serem extraídas (título, ano, autores, etc.) com detalhes específicos.

4. **Definição de Atributos:**
   - Especifica o formato e detalhes de cada atributo (e.g., "Author Contact" como lista de dicionários).

5. **Diretrizes:**
   - Regras para a extração (precisão, concisão, manejo de dados ausentes).

6. **Formato de Saída:**
   - Especifica que a resposta deve ser em formato JSON com chaves específicas para cada atributo.

In [ ]:
prompt_text = '''
Scientific research paper:
---
{document}
---

You are an expert in analyzing scientific research papers. Please carefully read the provided research paper above and extract the following key information:

Extract these six (6) properties from the research paper:
- Paper Title: The full title of the research paper
- Publication Year: The year the paper was published
- Authors: The full names of all authors of the paper
- Author Contact: A list of dictionaries, where each dictionary contains the following keys for each author:
  - Name: The full name of the author
  - Institution: The institutional affiliation of the author
  - Email: The email address of the author (if provided)
- Abstract: The full text of the paper's abstract
- Summary Abstract: A concise summary of the abstract in 2-3 sentences, highlighting the key points

Guidelines:
- The extracted information should be factual and accurate to the document.
- Be extremely concise, except for the Abstract which should be copied in full.
- The extracted entities should be self-contained and easily understood without the rest of the paper.
- If any property is missing from the paper, please leave the field empty rather than guessing.
- For the Summary Abstract, focus on the main objectives, methods, and key findings of the research.
- For Author Contact, create an entry for each author, even if some information is missing. If an email or institution is not provided for an author, leave that field empty in the dictionary.

Answer in JSON format. The JSON should contain 6 keys: "PaperTitle", "PublicationYear", "Authors", "AuthorContact", "Abstract", and "SummaryAbstract". The "AuthorContact" should be a list of dictionaries as described above.
'''

prompt_path = "./data/prompts/scientific_papers_prompt.txt"

os.makedirs("./data/prompts/", exist_ok=True)
with open(prompt_path, "w") as f:
    f.write(prompt_text)

# Exemplo de Uso

### Baixando artigo "Attention Is All You Need"

In [ ]:
import requests

# URL do PDF para download
url = "https://arxiv.org/pdf/1706.03762v7"

# Caminho para salvar o PDF baixado
pdf_path = "./data/1706.03762v7.pdf"

# Cria o diretório se ele não existir
os.makedirs(os.path.dirname(pdf_path), exist_ok=True)

# Faz o download do PDF
response = requests.get(url)

# Verifica se a requisição foi bem-sucedida
if response.status_code == 200:
    # Salva o PDF no caminho especificado
    with open(pdf_path, "wb") as pdf_file:
        pdf_file.write(response.content)
    print(f"PDF baixado e salvo em {pdf_path}")
else:
    print(f"Falha ao baixar o PDF. Código de status: {response.status_code}")

PDF baixado e salvo em ./data/1706.03762v7.pdf


### Chamada da Classe ResearchPaperProcessor

In [ ]:

# Inicializa o processador de artigos com o modelo desejado
processor = ResearchPaperProcessor(model_id='gpt-4o-mini')

# Define os caminhos para o PDF, prompt e pasta de saída
pdf_path = "./data/1706.03762v7.pdf"
prompt_path =  "./data/prompts/scientific_papers_prompt.txt"
output_folder = "./data/extracted_metadata"

# Cria a pasta de saída se ela não existir
os.makedirs(output_folder, exist_ok=True)

# Processa o artigo de pesquisa
processor.process_research_paper(pdf_path, prompt_path, output_folder)

Processando artigo de pesquisa: ./data/1706.03762v7.pdf
Conteúdo do texto extraído do PDF: ./data/1706.03762v7.pdf
Metadados extraídos usando gpt-4o-mini para ./data/1706.03762v7.pdf
Metadados salvos em ./data/extracted_metadata/1706.03762v7.json


### Visualizando JSON Resultante

In [ ]:
import json

# Define the path to the JSON file
file_path = './data/extracted_metadata/1706.03762v7.json'

# Read the JSON file
try:
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
        # Output the content of the JSON file
        print(json.dumps(data, indent=4))  # Pretty print the JSON data
except FileNotFoundError:
    print(f"The file at {file_path} was not found.")
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")
except Exception as e:
    print(f"An error occurred: {e}")

{
    "PaperTitle": "Attention Is All You Need",
    "PublicationYear": "2017",
    "Authors": [
        "Ashish Vaswani",
        "Noam Shazeer",
        "Niki Parmar",
        "Jakob Uszkoreit",
        "Llion Jones",
        "Aidan N. Gomez",
        "\u0141ukasz Kaiser",
        "Illia Polosukhin"
    ],
    "AuthorContact": [
        {
            "Name": "Ashish Vaswani",
            "Institution": "Google Brain",
            "Email": "avaswani@google.com"
        },
        {
            "Name": "Noam Shazeer",
            "Institution": "Google Brain",
            "Email": "noam@google.com"
        },
        {
            "Name": "Niki Parmar",
            "Institution": "Google Research",
            "Email": "nikip@google.com"
        },
        {
            "Name": "Jakob Uszkoreit",
            "Institution": "Google Research",
            "Email": "usz@google.com"
        },
        {
            "Name": "Llion Jones",
            "Institution": "Google Research",
      

[](./data/extracted_metadata/json_output.png)